# Plotting 1-D data

`scipp` offers a number of different ways to plot data from a `DataArray` or a `Dataset`. It uses the `matplotlib` graphing library to do so.

In [ ]:
import numpy as np
import scipp as sc
from scipp.plot import plot

## Basic plot

Plotting is done using the `plot` (or `scipp.plot.plot`) function.
Generally the information in a dataset is sufficient to produce a useful plot out of the box.

For example, a simple plot from a 1D dataset is produced as follows:

In [ ]:
d = sc.Dataset()
N = 50
d.coords['tof'] = sc.Variable(['tof'], values=np.arange(N).astype(np.float64),
                                unit=sc.units.us)
d['Sample'] = sc.Variable(['tof'], values=10.0*np.random.rand(N),
                          unit=sc.units.counts)
plot(d)

## With error bars

Error bars are shown automatically if variances are present in the data:

In [ ]:
d['Sample'].variances = np.square(np.random.rand(N))
plot(d)

Note that the length of the errors bars is the standard-deviation, i.e., the square root of the variances stored in the data.

## Multiple variables on the same axes

### Plotting a Dataset with multiple entries

If a dataset contains more than one 1D variable with the same coordinates, they are plotted on the same axes:

In [ ]:
d['Background'] = sc.Variable(['tof'], values=5.0*np.random.rand(N),
                              unit=sc.units.counts)
plot(d)

It is possible to hide the error bars with

In [ ]:
plot(d, errorbars=False)
plot(d, errorbars={'Sample': False})

We can always plot just a single item of the dataset:

In [ ]:
plot(d['Background'])

### Overplotting using a dict of DataArrays

One can also supply the `plot` function with a `dict` of data arrays, and compatible data arrays will be overplotted on the same axes:

In [ ]:
plot({"My sample": d["Sample"], "My background": d["Background"]})

This can become useful when wanting to plot slices of a 2D data array:

In [ ]:
M = 100
L = 5
xx = np.arange(M, dtype=np.float64)
yy = np.arange(L, dtype=np.float64)
x, y = np.meshgrid(xx, yy)
b = M/20.0
c = M/5.0
e = L/10.0
r = np.sqrt(((x-c)/b)**2 + (y/e)**2)
a = np.sin(r)
d2d = sc.Dataset()
d2d.coords['x'] = sc.Variable(['x'], values=xx, unit=sc.units.m)
d2d.coords['y'] = sc.Variable(['y'], values=yy, unit=sc.units.m)
d2d['Signal'] = sc.Variable(['y', 'x'], values=a, unit=sc.units.counts)
plot({f'slice-{i}': d2d['Signal']['y', i] for i in range(L)})

Or using the `collapse` helper function, which returns a `dict` of data arrays:

In [ ]:
plot(sc.collapse(d2d["Signal"], keep='x'))

## Customizing linestyles, markers and colors

Linestyles can be customized following the Matplotlib API.
For instance, it is possible to connect the dots by setting `linestyle='solid'`:

In [ ]:
plot(d, linestyle='solid')

Marker colors and symbols can be changed via the `color` and `marker` keyword arguments:

In [ ]:
plot(d, color=['red', '#30D5F9'], marker=['s', 'x'])

The supplied `color` and `marker` arguments can also be a list of integers, which correspond to one of the pre-defined [colors](https://matplotlib.org/3.1.1/users/dflt_style_changes.html) or [markers](https://matplotlib.org/3.1.1/api/markers_api.html) (which were taken from matplotlib). In addition, the grid can also be displayed:

In [ ]:
plot(d, color=[6, 8], grid=True)

## Logarithmic scales

Logarithmic axes are supported as follows:

In [ ]:
plot(d, logx=True)
plot(d, logy=True)
plot(d, logxy=True)

## Histograms
Histograms are automatically generated if the coordinate is bin edges:

In [ ]:
d['Histogram'] = sc.Variable(['tof'], values=20.0*np.random.rand(N-1),
                             unit=sc.units.counts)
plot(d['Histogram'])

and with error bars

In [ ]:
d['Histogram'].variances = 5.0*np.random.rand(N-1)
plot(d['Histogram'])

The histogram color can be customized:

In [ ]:
plot(d['Histogram'], color='#000000')

## Multiple 1D variables with different dimensions

`scipp.plot` also supports multiple 1-D variables with different dimensions (note that the data entries are grouped onto the same graph if they have the same dimension and unit):

In [ ]:
M = 60
d.coords['x'] = sc.Variable(['x'],
                              values=np.arange(M).astype(np.float64),
                              unit=sc.units.m)
d['OtherSample'] = sc.Variable(['x'], values=10.0*np.random.rand(M),
                                   unit=sc.units.s)
d['OtherNoise'] = sc.Variable(['x'], values=7.0*np.random.rand(M-1),
                                  variances=3.0*np.random.rand(M-1),
                                  unit=sc.units.s)
d['SomeKgs'] = sc.Variable(['x'], values=20.0*np.random.rand(M),
                                   unit=sc.units.kg)
plot(d)

## Custom labels along x axis

Sometimes one wishes to have `labels` along the X axis instead of the coordinate. This can be achieved via the `axes` keyword argument:

In [ ]:
d1 = sc.Dataset()
N = 100
d1.coords['tof'] = sc.Variable(['tof'],
                                 values=np.arange(N).astype(np.float64),
                                 unit=sc.units.us)
d1['Sample'] = sc.Variable(['tof'],
                           values=10.0 * np.random.rand(N),
                           unit=sc.units.counts)
d1.coords['somelabels'] = sc.Variable(['tof'],
                                      values=np.linspace(101., 105., N),
                                      unit=sc.units.s)
plot(d1, axes=['somelabels'])

If one has multiple entries in a `Dataset`, the labels corresponding to each dimension need to be specified in a dictionary-like fashion:

In [ ]:
M = 50
d1.coords['x'] = sc.Variable(['x'],
                             values=np.arange(M).astype(np.float64),
                             unit=sc.units.m)
d1['Sample2'] = sc.Variable(['x'],
                            values=10.0 * np.random.rand(M),
                            unit=sc.units.counts)
d1.coords['Xlabels'] = sc.Variable(['x'],
                                   values=np.linspace(151., 155., M),
                                   unit=sc.units.s)
plot(d1, axes={'x': 'Xlabels', 'tof': 'somelabels'})

## Plotting masks

If a dataset contains masks, the symbols of masks data points will have a thick black contour in a 1D plot:

In [ ]:
d4 = sc.Dataset()
N = 50
x = np.arange(N).astype(np.float64)
d4.coords['tof'] = sc.Variable(['tof'], values=x,
                                 unit=sc.units.us)
d4['Sample'] = sc.Variable(['tof'], values=3*np.sin(x/5)+3,
                           unit=sc.units.counts)
d4['Background'] = sc.Variable(['tof'], values=1.0*np.random.rand(N),
                               unit=sc.units.counts)
d4.masks['mask1'] = sc.Variable(['tof'],
                                values=np.where(np.abs(x-40) < 10, True, False))
plot(d4)

A toggle button below the plot can be used to hide/show the masks. The color of the masks can be changed as follows:

In [ ]:
plot(d4, masks={'color': 'red'})

### Masks on histograms

Masks on a histogram show up as a thick black line:

In [ ]:
d4 = sc.Dataset()
N = 50
x = np.arange(N+1).astype(np.float64)
d4.coords['tof'] = sc.Variable(['tof'], values=x,
                                 unit=sc.units.us)
d4['Histo'] = sc.Variable(['tof'], values=3*np.sin(x[:-1]/5)+3,
                           unit=sc.units.counts)
d4.masks['mask1'] = sc.Variable(['tof'],
                                values=np.where(np.abs(x[:-1]-40) < 10, True, False))
plot(d4)

## Saving figures
Static `pdf` or `png` copies of the figures can be saved to file (note that any buttons displayed under a figure are not saved to file). This is achieved as follows:

In [ ]:
plot(d4, filename='my_1d_figure.pdf')